<img src="https://user-images.githubusercontent.com/4486578/60554111-8fc27400-9d79-11e9-9ca7-6d78ee89ea70.png" width=50% align=right>

<h1>AIBECS.jl</h1>

*The ideal tool for exploring global marine biogeochemical cycles*

**A**lgebraic **I**mplicit **B**iogeochemical **E**lemental **C**ycling **S**ystem

Check it on GitHub (look for [AIBECS.jl](https://github.com/briochemc/AIBECS.jl))

<img src="https://pbs.twimg.com/profile_images/1829321548/ess_logo_400x400.png" width=10% align=right>
<img src="https://user-images.githubusercontent.com/4486578/61599460-6d32c500-ac6c-11e9-9796-0b8892f0342d.png" width=10% align=left>

<center>A <b>Julia</b> package developed by <b>Benoît Pasquier</b> at the Department of Earth System Sciences, <b>UCI</b></center>

# What is the AIBECS?

&#8594; A system to easily create models of marine biogeochemistry.

To build a BGC model with the AIBECS, you just need to

1. Specify the <span style="color:#389826">**local sources and sinks**</span>

2. Chose the <span style="color:#4063d8">**ocean circulation**</span>

3. Specify the <span style="color:#9558b2">**particle transport**</span> (for non-dissolved tracers if any)

<img src="https://user-images.githubusercontent.com/4486578/61606237-9a8f6b00-ac8c-11e9-8aa7-1267ab0f911a.png" width=90%>

#### Example 1: A radiocarbon model embedded in a 5-box-model circulation

<img src="https://user-images.githubusercontent.com/4486578/58314610-3b130b80-7e53-11e9-9fe8-9527cdcca2d0.png" width=80%>

(Credit: François Primeau and Louis Primeau)

##### The circulation

AIBECS provides the circulation as a sparse matrix

$$\nabla \cdot \left[ \boldsymbol{u} - \mathbf{K} \cdot \nabla \right] C \longrightarrow \mathbf{T} \, \boldsymbol{C}$$

Load the circulation via `load`:

In [ ]:
using AIBECS
wet3D, grd, T = Primeau_2x2x2.load();

`wet3D` is the mask of "wet" points

In [ ]:
wet3D

`grd` is the grid of the circulation

In [ ]:
grd

We can inspect `grd` a little...<br>
its size

In [ ]:
size(grd)

its boxes, one by one

In [ ]:
grd[4]

We can iterate on its boxes

In [ ]:
[println(box) for box in grd if (box.lat ≤ 0u"°") & (box.depth ≥ 1000u"m")];

We can check the depth of the boxes arranged in 3D

In [ ]:
grd.depth_3D

Or check their latitudes

In [ ]:
grd.lat_3D

`T` is the matrix of the circulation

In [ ]:
T

A sparse matrix is indexed by its non-zero values, but we can check it out in full

In [ ]:
Matrix(T)

<h5>Radiocarbon, a tracer label for water age</h5> 

<br>

<img src="https://wserv4.esc.cam.ac.uk/pastclimate/wp-content/uploads/2014/09/Radiocarbon-cycle_2.jpg" width=70% align=left>

*Image credit: Luke Skinner, University of Cambridge*




<h5>Radiocarbon, a tracer label for water age</h5> 

- **Tracer equation**
    ($x=$ Radiocarbon concentration)

    $$\frac{\partial x}{\partial t} + \nabla \cdot \left[ \boldsymbol{u} - \mathbf{K} \cdot \nabla \right] x = \Lambda(R_\mathsf{atm} - x) - x / \tau$$

    - $\Lambda(R_\mathsf{atm} - x) =$ air–sea exchange
    - $x / \tau=$ radioactive decay

- With the **transport matrix**, `T`

$$\frac{\partial \boldsymbol{x}}{\partial t} + \mathbf{T} \, \boldsymbol{x} = \mathbf{\Lambda}(R_\mathsf{atm} - \boldsymbol{x}) - \boldsymbol{x} / \tau.$$

##### Translating to AIBECS Code is easy

To use AIBECS, we recast the equations into the generic form of

$$\frac{\partial \boldsymbol{x}}{\partial t} + \mathbf{T}(\boldsymbol{p}) \, \boldsymbol{x} = \boldsymbol{G}(\boldsymbol{x}, \boldsymbol{p})$$

- $\mathbf{T}(\boldsymbol{p}) =$ is the transport operator (here the matrix `T`)
- $\boldsymbol{G}(\boldsymbol{x}, \boldsymbol{p}) = \mathbf{\Lambda}(R_\mathsf{atm} - \boldsymbol{x}) - \boldsymbol{x} / \tau$ local sources minus sinks
- $\boldsymbol{p} =$ vector of model parameters


In [ ]:
function G(x, p)
    τ, Ratm = p.τ, p.Ratm
    return Λ(Ratm .- x, p) - x / τ
end

Air–sea gas exchange:

In [ ]:
h = grd.δdepth[1] |> ustrip   # height of top layer
Λ(x, p) = p.λ / h * surface_boxes .* x

Vector of surface boxes

In [ ]:
iwet = findall(vec(wet3D))
surface_boxes = grd.depth_3D[iwet] .== grd.depth[1]

##### Creating model parameters with AIBECS is easy too

In [ ]:
t = empty_parameter_table()
add_parameter!(t, :τ, 5730u"yr"/log(2)) # radioactive decay e-folding timescale
add_parameter!(t, :λ, 50u"m" / 10u"yr") # piston velocity
add_parameter!(t, :Ratm, 1.0u"mol/m^3") # atmospheric concentration
t

Create `p`:

In [ ]:
initialize_Parameters_type(t, "C14_shoebox_parameters")
p = C14_shoebox_parameters()   

##### Generate the state function and its Jacobian in a one liner

$$\frac{\partial \boldsymbol{x}}{\partial t} = \boldsymbol{F}(\boldsymbol{x}, \boldsymbol{p}) = \boldsymbol{G}(\boldsymbol{x}, \boldsymbol{p}) - \mathbf{T}(\boldsymbol{p}) \, \boldsymbol{x} = \boldsymbol{F}(\boldsymbol{x}, \boldsymbol{p})$$

defines the rate of change of the state, $\boldsymbol{x}$


We generate `F` and `∇ₓF` via

In [ ]:
F, ∇ₓF = state_function_and_Jacobian(p -> T, G) ;

Let's try `F` on a random state vector `x`

In [ ]:
x = rand(length(iwet))
F(x,p)

And `∇ₓF`:

In [ ]:
Matrix(∇ₓF(x,p))

##### Time stepping is provided

AIBECS provides the [Crank-Nicolson](https://en.wikipedia.org/wiki/Crank%E2%80%93Nicolson_method) scheme.

A single step:

In [ ]:
δt = ustrip(1.0u"yr" |> u"s")
AIBECS.crank_nicolson_step(x, p, δt, F, ∇ₓF)

Let's track the evolution of `x` through time

In [ ]:
function time_steps(x₀, Δt, n, F, ∇ₓF)
    x_hist = [x₀]
    δt = Δt / n
    for i in 1:n
        push!(x_hist, AIBECS.crank_nicolson_step(last(x_hist), p, δt, F, ∇ₓF))
    end
    return reduce(hcat, x_hist), 0:δt:Δt
end

In [ ]:
Δt = 5000u"yr" |> u"s" |> ustrip
x₀ = ones(5)             
x_hist, t_hist = time_steps(x₀, Δt, 1000, F, ∇ₓF)

Make a plot

In [ ]:
using PyPlot, PyCall
C14age_hist = -log.(x_hist) * (p.τ * u"s" |> u"yr" |> ustrip)
figure(figsize=(8,4))
plot(t_hist .* 1u"s" .|> u"yr" .|> ustrip, C14age_hist')
xlabel("simulation time (years)")
ylabel("¹⁴C age (years)")
legend("box " .* string.(iwet))
title("Simulation of the evolution of ¹⁴C age with Crank-Nicolson time steps")

We chan check how **stable** the age is by tracking the norm of `F(x,p)`.
Specifically, $\frac{\|\boldsymbol{x}\|}{\|\boldsymbol{F}(\boldsymbol{x},\boldsymbol{p})\|}$ gives an **equilibrium timescale**:

In [ ]:
using LinearAlgebra
Δt = 40000u"yr" |> u"s" |> ustrip
x₀ = ones(5)             
x_hist, t_hist = time_steps(x₀, Δt, 4000, F, ∇ₓF)
figure(figsize=(7,3))
semilogy(t_hist .* 1u"s" .|> u"yr" .|> ustrip, [norm(x_hist[:,i]) / norm(F(x_hist[:,i],p)) * 1u"s" |> u"yr" |> ustrip for i in 1:size(x_hist,2)])
xlabel("simulation time (years)")
ylabel("Equilibirum timescale (years)")
title("Stability of ¹⁴C age with simulation time")

##### Solving directly for the steady state in two lines

Instead, we can directly solve for the **steady-state**, $\boldsymbol{s}$,

i.e., such that $\boldsymbol{F}(\boldsymbol{s},\boldsymbol{p}) = 0$:

In [ ]:
prob = SteadyStateProblem(F, ∇ₓF, x, p)
s = solve(prob, CTKAlg()).u

and check the equilibrium timescale at the steady-state solution, `s`

In [ ]:
norm(s) / norm(F(s,p)) * u"s" |> u"yr"

Solving directly for the steady state is **equivalent to ~35'000 years** of model time-steps!

# Using a *real* circulation

The Ocean Circulation Inverse Model (OCIM)

In [ ]:
wet3D, grd, T = AIBECS.OCIM1.load() ;

In [ ]:
F, ∇ₓF = state_function_and_Jacobian(p -> T, G)

In [ ]:
iwet = findall(vec(wet3D))
x = ones(size(iwet))

In [ ]:
p

Redefine `h` and `surface_boxes` for the new grid

In [ ]:
h = grd.δdepth[1] |> ustrip   
surface_boxes = grd.depth_3D[iwet] .== grd.depth[1]

In [ ]:
F(x,p)

In [ ]:
∇ₓF(x,p)

In [ ]:
prob = SteadyStateProblem(F, ∇ₓF, x, p)

In [ ]:
s = solve(prob, CTKAlg()).u

In [ ]:
C14age = -log.(s) * p.τ * u"s" .|> u"yr"

In [ ]:
C14age_3D = fill(NaN, size(grd))
C14age_3D[iwet] .= ustrip.(C14age)
levels = 0:100:2400
using GR, Interact
lon, lat = grd.lon |> ustrip, grd.lat |> ustrip
GR.figure(figsize=(10,5))
@manipulate for iz=1:24
    GR.xlim([0,360])
    GR.title("14C age at $(AIBECS.round(grd.depth[iz])) depth using the OCIM1 circulation")
    GR.contourf(lon, lat, C14age_3D[:,:,iz]', levels=levels)
end


In [ ]:
GR.figure(figsize=(10,5))
@manipulate for lon in (grd.lon .|> ustrip)
    GR.title("14C age at $(round(lon))° longitude using the OCIM1 circulation")
    ilon = findfirst(ustrip.(grd.lon) .== lon)
    GR.contourf(lat, reverse(-grd.depth .|> ustrip), reverse(C14age_3D[:,ilon,:], dims=2), levels=levels)
end